# Create Heat Map for significant Cell Cycle genes

This notebook looks at the significant genes in at least one cancer in the DNA Replication pathway. Pancancer heat maps are created with circle size showing significance and color showing differences in median.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import gseapy as gp
import re
import sys 

import cptac
import cptac.utils as u

import plot_utils as p

# Step 1: Run GSEA for significant genes in at least 1 cancer

First read in sig_pval_all_proteins.csv into a df. This csv file contains only genes with a significant p-value in at least one cancer. Then run GSEA using a list of genes from the df.

In [2]:
root = R'~\Github\WhenMutationsDontMatter\PTEN\Step_3_trans_effect\csv'
sig_df = pd.read_csv(root+R'\sig_pval_heatmap.csv')

prot_list = list(sig_df.Proteomics) # list of genes with a sig pval in >= 1 cancer
prot_enr = gp.enrichr(gene_list = prot_list, description='Tumor_partition', gene_sets='Reactome_2016', 
                       outdir='/Enrichr')

In [35]:
prot_enr.res2d.head(25)

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
0,Processing of Capped Intron-Containing Pre-mRN...,116/193,1.379053e-53,2.109952e-50,0,0,4.603878,560.363828,NUP107;NUP188;EIF4A3;HNRNPU;EFTUD2;SNRPD2;SNRP...,Reactome_2016
1,Gene Expression Homo sapiens R-HSA-74160,434/1631,1.734808e-53,1.327128e-50,0,0,2.038257,247.619892,TDRKH;RPL4;ATF2;MDC1;NUP107;HNRNPU;EHMT1;PHAX;...,Reactome_2016
2,mRNA Splicing - Major Pathway Homo sapiens R-H...,85/134,3.384052e-42,1.725866e-39,0,0,4.858892,463.973174,EIF4A3;HNRNPU;YBX1;PRPF19;USP39;ELAVL1;EFTUD2;...,Reactome_2016
3,mRNA Splicing Homo sapiens R-HSA-72172,85/144,1.081367e-38,4.136230e-36,0,0,4.521469,395.266853,EIF4A3;HNRNPU;YBX1;PRPF19;USP39;ELAVL1;EFTUD2;...,Reactome_2016
4,Infectious disease Homo sapiens R-HSA-5663205,123/348,8.726101e-27,2.670187e-24,0,0,2.707379,162.452135,RPL4;NUP107;NUP188;RPL10A;RPL9;PSMD8;RPS15;PSM...,Reactome_2016
5,Major pathway of rRNA processing in the nucleo...,78/166,1.231200e-26,3.139560e-24,0,0,3.599230,214.727285,LTV1;RPL4;DDX47;RPP30;WDR3;RPLP0;PWP2;RPL10A;R...,Reactome_2016
6,rRNA processing Homo sapiens R-HSA-72312,81/180,5.024648e-26,1.098244e-23,0,0,3.446955,200.794988,LTV1;RPL4;DDX47;RPP30;WDR3;RPLP0;NAT10;PWP2;RP...,Reactome_2016
7,"Cell Cycle, Mitotic Homo sapiens R-HSA-69278",143/462,2.644457e-24,5.057524e-22,0,0,2.370922,128.716327,ANKLE2;NUP107;NUP188;ZWILCH;BUB1B;PPP2R2A;SMC4...,Reactome_2016
8,Transport of Mature Transcript to Cytoplasm Ho...,46/74,6.835673e-23,1.162064e-20,0,0,4.761560,243.017163,NUP205;NUP107;SEH1L;NUP188;EIF4A3;NUP160;NXF1;...,Reactome_2016
9,Cell Cycle Homo sapiens R-HSA-1640170,160/566,1.870504e-22,2.861870e-20,0,0,2.165343,108.333543,MDC1;ANKLE2;NUP107;NUP188;ZWILCH;BUB1B;PPP2R2A...,Reactome_2016


# Step 2: Get the list of significant genes 

In [36]:
index = 19
trans = prot_enr.res2d.Genes[index]
genes = trans.split(';')
print(prot_enr.res2d.Term[index])
print('total genes:',len(genes))

DNA strand elongation Homo sapiens R-HSA-69190
total genes: 25


# Step 3: Create HeatMap

Slice out genes from the DNA Replication pathway from the df with genes sig in > 1 cancer.

In [37]:
# sig > 1 cancer
bool_df = sig_df.Proteomics.isin(genes)
plot_df = sig_df[bool_df]
len(plot_df.Proteomics.unique())

25

In [38]:
# Only include p-values a certain cutoff
a = 0.01
plot_df = plot_df.loc[plot_df['P_Value'] <= a]

In [39]:
p.plotCircleHeatMap(plot_df, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=400, plot_width= 800)

Loading BokehJS ...

# Step 4: Create a HeatMap with both +/- differences in median

 Read in the pos_neg_df.csv to create a df with only genes that have a pos and neg difference in median in different cancers. Slice out the genes that have a pos and neg difference in median in the pathway using the list of genes with a significant p-value in the pathway. 

In [8]:
pos_neg_df = pd.read_csv(root+R'\pos_neg_df.csv')

In [40]:
get = pos_neg_df.Proteomics.isin(genes) # bool df where True has both pos and neg
genes_pn = pos_neg_df[get] # Keep only genes with pos and neg
genes_pn.Proteomics.unique()

array(['MCM6', 'MCM4', 'GINS3', 'RFC5', 'MCM5', 'MCM2', 'MCM7', 'MCM3',
       'RFC2', 'RFC4', 'GINS4', 'POLA2', 'GINS2'], dtype=object)

In [41]:
# Only include p-values < a certain cutoff
genes_pn = genes_pn.loc[genes_pn['P_Value'] <= a]

In [42]:
p.plotCircleHeatMap(genes_pn, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=400, plot_width=600)

Loading BokehJS ...

# Sig in multiple cancers

In [12]:
mult_sig_df = pd.read_csv(root+R'\mult_sig_pval_heatmap.csv')

mult_sig_list = list(mult_sig_df.Proteomics.unique()) # list of genes with a sig pval in > 1 cancer
enr2 = gp.enrichr(gene_list = mult_sig_list, description='Tumor_partition', gene_sets='Reactome_2016', 
                       outdir='/Enrichr')

In [13]:
len(mult_sig_df.Proteomics.unique())

311

In [14]:
enr2.res2d.head()

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
0,Gene Expression Homo sapiens R-HSA-74160,74/1631,2.081067e-17,3.184033e-14,0,0,2.917745,112.073703,WDR3;WDR4;PTEN;HNRNPU;NAT10;ZC3H8;ADAR;PPP2R2A...,Reactome_2016
1,"Cell Cycle, Mitotic Homo sapiens R-HSA-69278",37/462,2.789117e-16,2.133675e-13,0,0,5.150262,184.459924,TOP2A;AHCTF1;PCNA;MCM7;DCTN1;BUB1B;PPP2R2A;ORC...,Reactome_2016
2,Cell Cycle Homo sapiens R-HSA-1640170,38/566,3.204908e-14,1.634503e-11,0,0,4.317544,134.152603,TOP2A;AHCTF1;PCNA;MCM7;DCTN1;BUB1B;PPP2R2A;ORC...,Reactome_2016
3,Processing of Capped Intron-Containing Pre-mRN...,23/193,3.858260e-14,1.475784e-11,0,0,7.663729,236.701740,RANBP2;DDX5;NUP210;CPSF1;NUP155;NCBP1;NUP133;D...,Reactome_2016
4,DNA strand elongation Homo sapiens R-HSA-69190,11/32,1.039399e-12,3.180562e-10,0,0,22.106109,609.960125,GINS2;RFC3;PCNA;RFC4;MCM7;RFC2;GINS4;MCM3;MCM4...,Reactome_2016


In [28]:
index2 = 4
trans2 = enr2.res2d.Genes[index2]
genes_mult = trans2.split(';')
print(enr2.res2d.Term[index2])
print('total genes:',len(genes_mult))

DNA strand elongation Homo sapiens R-HSA-69190
total genes: 11


In [29]:
bool_df = mult_sig_df.Proteomics.isin(genes_mult)
plot_df2 = mult_sig_df[bool_df]
plot_df2.Proteomics.unique()

array(['MCM4', 'PCNA', 'MCM5', 'MCM2', 'MCM7', 'MCM3', 'RFC3', 'RFC2',
       'RFC4', 'GINS4', 'GINS2'], dtype=object)

In [30]:
# Only include p-values < a certain cutoff
plot_df2 = plot_df2.loc[plot_df2['P_Value'] <= a]

In [31]:
p.plotCircleHeatMap(plot_df2, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=300, plot_width=650)

Loading BokehJS ...

In [32]:
get = pos_neg_df.Proteomics.isin(genes_mult) # bool df where True has both pos and neg
genes_pn_mult = pos_neg_df[get] # Keep only genes with pos and neg
genes_pn_mult.Proteomics.unique()

array(['MCM4', 'MCM5', 'MCM2', 'MCM7', 'MCM3', 'RFC2', 'RFC4', 'GINS4',
       'GINS2'], dtype=object)

In [33]:
# Only include p-values < a certain cutoff
genes_pn_mult = genes_pn_mult.loc[genes_pn_mult['P_Value'] <= a]

In [34]:
p.plotCircleHeatMap(genes_pn_mult, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=400, plot_width=600)

Loading BokehJS ...

# Check other pathways the genes (+/- and sig in mult. cancers) are part of

In [22]:
gnm = list(genes_pn_mult.Proteomics.unique())
print(gnm)
enr3 = gp.enrichr(gene_list = gnm, description='Tumor_partition', gene_sets='KEGG_2016', 
                       outdir='/Enrichr')

['WDR75', 'HEATR1', 'NAT10', 'DCAF13', 'WDR46', 'IMP3', 'UTP6', 'MPHOSPH10']


In [23]:
enr3.res2d.head()

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
0,Ribosome biogenesis in eukaryotes Homo sapiens...,6/89,1.818101e-13,5.327036e-11,0,0,168.539326,4944.238268,UTP6;IMP3;HEATR1;NAT10;MPHOSPH10;WDR75,KEGG_2016


In [24]:
# Look at p-values 
certain_gene = 'GINS4'
all_df = pd.read_csv(root+R'\all_heatmap.csv')
gene_df = all_df.loc[all_df['Proteomics'] == certain_gene]
gene_df

,Proteomics,P_Value,Medians,Cancer
1005,GINS4,0.001694,0.742086,Gbm
11051,GINS4,0.000016,0.472815,Hnscc
24844,GINS4,0.116370,0.845900,Luad
40340,GINS4,0.645872,0.308800,Lscc
44834,GINS4,0.007736,0.679200,Brca
58444,GINS4,0.350344,-0.049301,Ov
67385,GINS4,0.051559,-0.325000,En
82767,GINS4,0.873907,0.168000,Colon
